<a href="https://colab.research.google.com/github/joe-jachim/pytorch-playground/blob/main/pytorch-lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pytorch-lightning

     |████████████████████████████████| 675kB 4.9MB/s 
     |████████████████████████████████| 276kB 9.5MB/s 
     |████████████████████████████████| 829kB 13.4MB/s 
     |████████████████████████████████| 102kB 7.4MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=6302eec5713d13bb0a3bd617f4f2d7b6c21390747a66d93097ec01e8461e7622
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=2b34ac75ee62d5e5169755f884e39306ee40711d64dab73029ffc89de9609379
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built PyYAML future
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninst

In [2]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import pytorch_lightning as pl

In [3]:
class ResNet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28 * 28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)

    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h2 + h1)
    logits = self.l3(do)
    return logits

  def configure_optimizers(self):
    optimiser = optim.SGD(self.parameters(), lr=1e-2)
    return optimiser

  def training_step(self, batch, batch_idx):
    x, y = batch

    x = x.view(x.size(0), -1)

    logits = self(x)

    J = self.loss(logits, y)

    return {'loss': J}

  def train_dataloader(self):
    train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
    #train, val = random_split(train_data, [55000, 5000])
    train_loader = DataLoader(train_data, batch_size=32)
    #val_loader = DataLoader(val, batch_size=32)
    return train_loader

In [5]:
model = ResNet()

In [ ]:
trainer = pl.Trainer(progress_bar_refresh_rate = 20)
trainer.fit(model)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name | Type             | Params
------------------------------------------
0 | l1   | Linear           | 50.2 K
1 | l2   | Linear           | 4.2 K 
2 | l3   | Linear           | 650   
3 | do   | Dropout          | 0     
4 | loss | CrossEntropyLoss | 0     
------------------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params
